In [1]:
# Packages
import tsplib95
import random
import matplotlib.pyplot as plt
import os
import gzip

URV                                                                            MESIIA

Neural and Evolutionary Computation (NEC)
Assignment 4: Optimization with Genetic Algorithm

Teachers: Dr. Jordi Duch, Dr. Sergio Gomez

Student: Natzaret Gálvez Rísquez

Implementing a genetic algorithm for the Traveling Salesman Problem (TSP) using the TSPLIB library 

In [2]:
# File paths
tsp_file_path = 'C:/Users/Gari/Desktop/Assignments_NEC/A4/eil101.tsp'

# Load the dataset
dataset = tsplib95.load(tsp_file_path)

Genetic algorithm  for solving the Traveling Salesman Problem (TSP) using permutation representation and various selection, mutation, and crossover techniques

In [9]:
import numpy as np

# Function to calculate the total distance of a tour
def total_distance(tour, distances):
    total = 0
    for i in range(len(tour) - 1):
        total += distances[tour[i], tour[i + 1]]
    total += distances[tour[-1], tour[0]]  # Return to the starting city
    return total

# Function to initialize the population
def initialize_population(pop_size, num_cities):
    population = []
    for _ in range(pop_size):
        chromosome = np.random.permutation(num_cities)
        population.append(chromosome)
    return population

# Selection: Roulette Wheel Selection
def roulette_wheel_selection(population, distances):
    fitness_values = [1 / total_distance(chromosome, distances) for chromosome in population]
    total_fitness = sum(fitness_values)
    probabilities = [fitness / total_fitness for fitness in fitness_values]
    selected_indices = np.random.choice(len(population), 2, replace=False, p=probabilities)
    return [population[i] for i in selected_indices]

# Selection: Tournament Selection
def tournament_selection(population, distances, tournament_size=3):
    tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
    tournament = [population[i] for i in tournament_indices]
    best_chromosome = min(tournament, key=lambda x: total_distance(x, distances))
    return best_chromosome

# Crossover: Order Crossover (OX)
def order_crossover(parent1, parent2):
    n = len(parent1)
    start, end = sorted(np.random.choice(n, 2, replace=False))
    child = [-1] * n
    child[start:end + 1] = parent1[start:end + 1]
    remaining = [gene for gene in parent2 if gene not in child]
    index = 0
    for i in range(n):
        if child[i] == -1:
            child[i] = remaining[index]
            index += 1
    return child

# Crossover: Partially Mapped Crossover (PMX)
def partially_mapped_crossover(parent1, parent2):
    n = len(parent1)
    start, end = sorted(np.random.choice(n, 2, replace=False))
    child1, child2 = [-1] * n, [-1] * n
    child1[start:end + 1] = parent1[start:end + 1]
    child2[start:end + 1] = parent2[start:end + 1]
    
    for i in range(n):
        if parent2[i] not in child1:
            index = i
            while child1[index] != -1:
                index = parent1.index(parent2[index])
            child1[index] = parent2[i]
        if parent1[i] not in child2:
            index = i
            while child2[index] != -1:
                index = parent2.index(parent1[index])
            child2[index] = parent1[i]
    return child1, child2

# Mutation: Swap Mutation
def swap_mutation(chromosome):
    idx1, idx2 = np.random.choice(len(chromosome), 2, replace=False)
    chromosome[idx1], chromosome[idx2] = chromosome[idx2], chromosome[idx1]
    return chromosome

# Mutation: Inversion Mutation
def inversion_mutation(chromosome):
    start, end = sorted(np.random.choice(len(chromosome), 2, replace=False))
    chromosome[start:end + 1] = chromosome[start:end + 1][::-1]
    return chromosome







ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [8]:
# Genetic Algorithm
def genetic_algorithm(distances, population_size=50, generations=100, mutation_rate=0.1):
    num_cities = distances.shape[0]
    population = initialize_population(population_size, num_cities)
    for generation in range(generations):
        # Selection
        selected_parents = [roulette_wheel_selection(population, distances) for _ in range(2)]
        # Crossover
        child1, child2 = partially_mapped_crossover(selected_parents[0], selected_parents[1])
        # Mutation
        if np.random.rand() < mutation_rate:
            child1 = swap_mutation(child1)
        if np.random.rand() < mutation_rate:
            child2 = inversion_mutation(child2)
        # Replace the worst individuals with the children
        population.remove(max(population, key=lambda x: total_distance(x, distances)))
        population.remove(max(population, key=lambda x: total_distance(x, distances)))
        population.extend([child1, child2])
    # Return the best solution found
    return min(population, key=lambda x: total_distance(x, distances))

# small dataset to try
if __name__ == "__main__":
    # Example distance matrix
    distances = np.array([[0, 10, 15, 20],
                          [10, 0, 35, 25],
                          [15, 35, 0, 30],
                          [20, 25, 30, 0]])
    solution = genetic_algorithm(distances)
    print("Best tour:", solution)
    print("Total distance:", total_distance(solution, distances))


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()